In [ ]:
import os
import cv2
import numpy as np
import random

In [ ]:
# To make sure that augmented images is not generated from validation object/background
VALIDATION_PATH = r'D:\Resources\Inat_Partial\Aves_Small_SS1_Validation\CV_0'

#OBJECT_PATH = r'D:\Resources\Inat_Partial\Aves_Small_SS1_Object'
OBJECT_PATH = r'D:\Resources\Inat_Partial\Aves_Small_SS1_Object_Dummy'
BACKGROUND_PATH = r'D:\Resources\Inat_Partial\Aves_Small_SS1_Background'
OUTPUT_PATH = r'D:\Resources\Inat_Partial\Aves_Small_SS1_Augmented'

# Whether to only use the same class backgrounds, otherwise augmented images will be generated from all subclasses
SAME_CLASS_ONLY = True
# Number of background image used per class for each object image
# so if NUM_BACKGROUND_PER_CLASS = 1, there will be n_class number of augmented images for each object
NUM_BACKGROUND_PER_CLASS = 5
# Whether to select random background image, otherwise the first non-matching background will be selected
IS_RANDOM_BACKGROUND = True #TODO: Non-random is not yet implemented
# Random pixel translation to shift the object away from the center of the background
# 100 means it can shift at most 100 pixels to any direction
RANDOM_PIXEL_SHIFT = 100
# Whether to horizontally flip the object randomly during the augmentation
RANDOM_HFLIP = True

In [ ]:
if os.path.isdir(OUTPUT_PATH) == False:
    os.mkdir(OUTPUT_PATH)

In [ ]:
# Set of filenames not to be used as background or foreground for each subdir
restricted_files = dict()

for subdir_name in os.listdir(VALIDATION_PATH):
    subdir_path = os.path.join(VALIDATION_PATH, subdir_name)
    restricted_files[subdir_name] = set()
    for image_name in os.listdir(subdir_path):
        restricted_files[subdir_name].add(image_name.split('.')[0])

In [ ]:
def generate_augmented_image(ob_path, bg_path, out_path, pixel_shift=0, hflip=False):
    bg = cv2.imread(bg_path, cv2.IMREAD_UNCHANGED)
    ob = cv2.imread(ob_path, cv2.IMREAD_UNCHANGED)
    
    # Get the top left of bounding box coordinate on the background to be replaced with the object
    # Object should be centered in the background, with possibly a little translation
    # Just naming them so they are easier to imagine
    bg_center_x = bg.shape[1] // 2
    bg_center_y = bg.shape[0] // 2
    bg_width = bg.shape[1]
    bg_height = bg.shape[0]
    ob_width = ob.shape[1]
    ob_height = ob.shape[0]
    
    # Make sure that the object is not bigger than the background in any dimension
    if ob_width > bg_width or ob_height > bg_height:
        return False

    shift_x = random.randint(-pixel_shift,pixel_shift)
    box_topleft_x = bg_center_x - (ob_width//2) + shift_x
    # Readjust the value if they go beyond the limit (0,bg_width-1) and (0,bg_height-1)
    box_topleft_x = 0 if box_topleft_x < 0 else box_topleft_x
    if box_topleft_x + ob_width > bg_width - 1:
        excess_width = (box_topleft_x + ob_width) - (bg_width - 1)
        box_topleft_x -= excess_width
    # Used to horizontally flip the object
    box_botright_x =box_topleft_x + ob_width
    
    shift_y = random.randint(-pixel_shift,pixel_shift)
    box_topleft_y = bg_center_y - (ob_height//2) + shift_y
    # Readjust the value if they go beyond the limit (0,bg_width-1) and (0,bg_height-1)
    box_topleft_y = 0 if box_topleft_y < 0 else box_topleft_y
    if box_topleft_y + ob_height > bg_height - 1:
        excess_height = (box_topleft_y + ob_height) - (bg_height - 1)
        box_topleft_y -= excess_height
        
    # Replace the background pixel with the object pixel if alpha > 0
    obj_pixels_y, obj_pixels_x = np.where(ob[:,:,3] > 0)
    for py,px in zip(obj_pixels_y, obj_pixels_x):
        bg_x = box_botright_x-px if hflip else box_topleft_x+px
        bg_y = box_topleft_y+py
        bg[bg_y, bg_x] = ob[py,px,:3]
    cv2.imwrite(out_path, bg)
    del bg,ob
    return True

In [ ]:
# To minimize the number of directory parsing
bg_subdir_images = {}
for bg_subdir_name in os.listdir(BACKGROUND_PATH):
    bg_subdir_path = os.path.join(BACKGROUND_PATH, bg_subdir_name)
    bg_subdir_images[bg_subdir_name] = set()
    for bg_image_name in os.listdir(bg_subdir_path):
        if bg_image_name.split('.')[0] not in restricted_files[bg_subdir_name]:
            bg_subdir_images[bg_subdir_name].add(bg_image_name)

# Start the loop from the object images
for obj_subdir_name in os.listdir(OBJECT_PATH):
    print('Augmenting',obj_subdir_name)
    obj_subdir_path = os.path.join(OBJECT_PATH, obj_subdir_name)
    output_subdir_path = os.path.join(OUTPUT_PATH, obj_subdir_name)
    if os.path.isdir(output_subdir_path) == False:
        os.mkdir(output_subdir_path)
    for obj_image_name in os.listdir(obj_subdir_path):
        # check for restricted files
        # object image follow the naming scheme of $imagename_index.png, so split based on _ instead
        obj_image_name_ori = obj_image_name.split('_')[0]
        if obj_image_name_ori in restricted_files[obj_subdir_name]:
            continue
            
        obj_image_path = os.path.join(obj_subdir_path, obj_image_name)
        
        # Whether to generate augmented images from same class only, constant configured beforehand
        bg_subdir_names = [obj_subdir_name] if SAME_CLASS_ONLY else list(bg_subdir_images.keys())
        for bg_subdir_name in bg_subdir_names:
            augmented_count = 0
            # To monitor backgrounds which are still available for augmentation
            available_bgs = set(bg_subdir_images[bg_subdir_name])
            while augmented_count < NUM_BACKGROUND_PER_CLASS and len(available_bgs) > 0:  
                # Sample random background from the pool
                random_bg_image_name = random.sample(available_bgs, 1)[0]
                # remove the selected background from the available backgrounds pool
                available_bgs.remove(random_bg_image_name)
                
                # make sure that the background is not the same as the object
                if obj_image_name_ori == random_bg_image_name.split('.')[0]:
                    continue
                    
                random_bg_image_path = os.path.join(BACKGROUND_PATH, bg_subdir_name, random_bg_image_name)
                
                # Output image name is $objname_$bgname.jpg
                output_image_path = os.path.join(output_subdir_path, 
                                                 obj_image_name_ori + '_' + random_bg_image_name)
                # Randomly flip the object depending on the setting and generated random value
                to_flip = True if random.random() > 0.5 and RANDOM_HFLIP else False
                succ = generate_augmented_image(obj_image_path, 
                                                random_bg_image_path, 
                                                output_image_path, 
                                                RANDOM_PIXEL_SHIFT, 
                                                to_flip)
                if succ:
                    augmented_count += 1